In [24]:
## Load Dataframe
import pandas as pd
import warnings
import fnmatch
import os

pd.set_option('future.no_silent_downcasting', True)
root = "assmtData"
def loadDataFrameFromFileRegex(root, regex):
    df_arr = []
    for path, subdirs, files in os.walk(root):
        for name in files:
            if fnmatch.fnmatch(name, regex) and os.path.getsize(os.path.join(path, name)) > 0:
                # print(os.path.join(path, name))
                df = pd.read_csv(os.path.join(path, name))
                df_arr.append(df)
    if not df_arr:
        warnings.warn("No matching file found in "+root+" for regex: "+regex+". Empty dataframe will be returned." )
        return pd.DataFrame()    
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=FutureWarning)      
        return pd.concat(df_arr, ignore_index=True)
        
df = loadDataFrameFromFileRegex(root, 'ASSESS*.csv')
a = df['action'].unique()
print(a)
# display(df[df['action'].isna()])


df = df.groupby(['tenant','assessment_id', 'action']).agg(first=('timestamp', 'min'),last=('timestamp', 'max')).reset_index()
df = df.pivot(index=['tenant','assessment_id'], columns='action', values=['first','last']).reset_index()
df.columns = [' '.join(col).strip() for col in df.columns.values]
# # display(df)

df['created_date']=df[['first start', 'first start/v2']].min(axis=1)
df['published_date']=df[['first /publish']]
df['last_submit_date']=df[['last /submit_all']]
df['last_review_date']=df[['last /review_all']]
df['last_reassessment_date']=df[['last /reassessment/now']]


df = df[['tenant','assessment_id','created_date', 'published_date', 'last_submit_date', 'last_review_date', 'last_reassessment_date']]
df['created_date']=(pd.to_datetime(df['created_date'],unit='ms')) 
df['published_date']=(pd.to_datetime(df['published_date'],unit='ms')) 

df.to_csv("assmt_report.csv")
display(df)

['/submit_all' '/review_all' '/reassessment/now' '/publish' 'start'
 'start/v2']


,tenant,assessment_id,created_date,published_date,last_submit_date,last_review_date,last_reassessment_date
0,tenant005529__assessmentautomationcom,811,NaT,NaT,1.716653e+12,NaN,NaN
1,tenant005529__assessmentautomationcom,812,NaT,NaT,1.716653e+12,NaN,NaN
2,tenant005529__assessmentautomationcom,814,NaT,NaT,NaN,NaN,1.717131e+12
3,tenant005529__assessmentautomationcom,815,NaT,NaT,1.717131e+12,1.717131e+12,1.717131e+12
4,tenant005529__assessmentautomationcom,818,NaT,NaT,1.717747e+12,NaN,NaN
...,...,...,...,...,...,...,...
11403,tenant995801__oxiquimicacombr,247,NaT,NaT,1.719947e+12,NaN,NaN
11404,tenant995801__oxiquimicacombr,306,NaT,NaT,1.718217e+12,1.718217e+12,NaN
11405,tenant995801__oxiquimicacombr,314,NaT,NaT,1.719951e+12,NaN,NaN
11406,tenant995801__oxiquimicacombr,330,NaT,NaT,1.719948e+12,NaN,NaN
